In [2]:
import Levenshtein

def error_rate(a, b):
    msd = Levenshtein.distance(a,b)
    
    return 100.0*msd/max(len(a), len(b))

In [3]:
import numpy as np
import dataIO
from sklearn import cross_validation, linear_model
reload(dataIO)

def learn_offset(points, targets):
    regr = linear_model.LinearRegression()
    regr.fit(points, targets)
    
    return regr

def run(userId, touches):
    locations, bod, targets_x, targets_y, y, touch_centers = dataIO.process_twohand(userId)

    locations = np.array(locations)
    targets_x = np.array(targets_x)
    targets_y = np.array(targets_y)

    regr_x = learn_offset(locations, targets_x)
    regr_y = learn_offset(locations, targets_y)

    new_points = []
    for i in range(len(touches)):
        point = touches[i]

        pred_x = regr_x.predict(point)
        pred_y = regr_y.predict(point)

        new_points.append([point[0]+pred_x, point[1]+pred_y])

    return np.array(new_points)

    

In [20]:
import numpy as np
import ast
import dataIO
reload(dataIO)

userId = 11
locations, bod, targets_x, targets_y, y, touch_centers = dataIO.process_twohand(userId)

locations = np.array(locations)
targets_x = np.array(targets_x)
targets_y = np.array(targets_y)

regr_x = learn_offset(locations, targets_x)
regr_y = learn_offset(locations, targets_y)

with open("/home/dimitar/Desktop/Python/Level5Project/Loggingapp/dataset.txt") as f:
    pool = f.read().splitlines()

#def read(userId):
req_sentences = []
typed_sentences = []
pred_sentences = []
#for i in range(13, 25):
for i in [15, 17, 21]:
    column_index = 0
    if i in [15, 17, 21]:
        column_index = 1

    with open("/home/dimitar/Desktop/Python/experiment/results/"
               +str(userId)+"_"+str(i)+"up.txt") as f:
        lines = f.read().splitlines()
        touches = map(lambda x: x.split('\t'), lines[1:])

    req_string = ''.join(np.array(touches)[:,column_index]) 
    locations = map(lambda x: list(ast.literal_eval(x)),
                            np.array(touches)[:,(column_index+2)])
   
    for sentence in pool: 
        index = req_string.find(sentence)
        if index!=-1:
            req_sentences.append(sentence)
            typed_locations = np.array(locations[index:index+len(sentence)])
            
            pred_x = regr_x.predict(typed_locations)
            pred_y = regr_y.predict(typed_locations)
            pred = typed_locations + np.dstack((pred_x, pred_y))[0]
            
            typed_sentences.append(dataIO.typed_string(typed_locations))
            pred_sentences.append(dataIO.typed_string(pred))

error_typed = []
error_pred = []
for i in range(len(req_sentences)):    
    req_sentence = req_sentences[i]
    typed_sentence = typed_sentences[i]
    pred_sentence = pred_sentences[i]
    error_typed.append(error_rate(req_sentence, typed_sentence))
    error_pred.append(error_rate(req_sentence, pred_sentence))

print
print ("Mean error rate before: %.2f %%"% np.mean(np.array(error_typed)))
print ("Mean error rate after: %.2f %%"% np.mean(np.array(error_pred)))


Mean error rate before: 5.91 %
Mean error rate after: 7.85 %
